In [1]:
import pandas as pd
import numpy as np
import openpyxl
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
TOP = Path.cwd().as_posix().replace('notebooks','')
raw_dir = Path(TOP) / 'data' /'raw'
processed_dir =  Path(TOP) / 'data' /'processed'
external_dir = Path(TOP)/'data'/'external'
interim_dir = Path(TOP)/'data'/'interim'

In [216]:
SI1 = pd.read_excel(external_dir/'Roe_sup4.xlsx', sheet_name = 'Evaluation of the RA hypotheses')

In [217]:
SI1.Scope.unique()

array(['Testing Proposal', nan], dtype=object)

In [218]:
[col for col in SI1.columns if 'AE' in col]

['*AE1',
 '*AE2',
 '*AE3',
 '*AE4',
 '*AE5',
 '*AE6',
 '*AE7',
 '*AE8',
 '*AE9',
 '*AE10',
 '*AE11',
 '*AE12',
 '*AE13',
 '*AE14',
 '*AE15',
 '*AE16',
 '*AE17']

In [219]:
SI1['*AE8'].replace(r'\n', '', regex = True)

0       
1       
2       
3       
4       
      ..
309    1
310    1
311    1
312     
313     
Name: *AE8, Length: 314, dtype: object

In [221]:
df1 = (
    SI1.assign(
        **{
            col: SI1[col].map(lambda x: str(x).replace('\n', ' ')).replace(' ', np.nan).astype(float) for col in SI1.loc[:, '*AE8':].columns
        }
    )

    .rename(
        columns={col: col.strip('*') for col in SI1.columns if 'AE' in col}
    )
)

In [222]:
df1.shape

(314, 61)

In [232]:
65+103+84+52+10

314

In [244]:
df1 = (df1
 .assign(RA_Present = lambda df: df['RA_Present'].replace('y', 'Y'))
  .assign(RA_Accepted = lambda df: df['RA_Accepted'].replace({'y':'Y', 'Y ': 'Y', 'N ': 'N'}))
)


In [256]:
df1.groupby(['RA_Accepted', '*RA_Type']).agg({'Target_Substance':'count'})

Target_Substance
RA_Accepted *RA_Type                  
N           Analogue               103
            Group                   52
            NR                      10
Y           Analogue                65
            Group                   84

In [262]:
df1 = (df1
 .rename(columns = {'*RA_Type': 'RA_Type'})
)

In [327]:
df1 = (df1
 .assign(index_key=lambda x: 'RA_' + (x.index + 1).astype(str))

)

In [328]:
df2 = (df1
 .query("RA_Type != 'NR'")
)

In [329]:
accepted = (df2
 .query("RA_Accepted == 'Y'")
)

In [331]:
test = (accepted
 .filter(['index_key','Target_Substance', 'EC_List_Number', 'CASRN', 'Target_Substance_Type', '*Source_Substance_EC_1',
       '*Source_Substance_Type_1', '*Source_Substance_EC_2',
       '*Source_Substance_Type_2', '*Source_Substance_EC_3',
       '*Source_Substance_Type_3', '*Source_Substance_EC_4',
       '*Source_Substance_Type_4', '*Source_Substance_EC_5',
       '*Source_Substance_Type_5', '*Source_Substance_EC_6',
       '*Source_Substance_Type_6', 'RA_Type'])
)

In [332]:
test.groupby('RA_Type').agg({'Target_Substance': 'count'})

,Target_Substance
RA_Type,
Analogue,65
Group,84


In [333]:
test[test['RA_Type'] =='Group']

,index_key,Target_Substance,EC_List_Number,CASRN,Target_Substance_Type,*Source_Substance_EC_1,*Source_Substance_Type_1,*Source_Substance_EC_2,*Source_Substance_Type_2,*Source_Substance_EC_3,*Source_Substance_Type_3,*Source_Substance_EC_4,*Source_Substance_Type_4,*Source_Substance_EC_5,*Source_Substance_Type_5,*Source_Substance_EC_6,*Source_Substance_Type_6,RA_Type
1,RA_2,Cobalt,231-158-0,7440-48-4,Mono-,231-589-4,Mono-,215-157-2,Mono-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group
2,RA_3,Cobalt oxide,215-154-6,1307-96-6,Mono-,231-589-4,Mono-,215-157-2,Mono-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group
3,RA_4,Cobalt sulphate,233-334-2,10124-43-3,Mono-,231-589-4,Mono-,215-157-2,Mono-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group
9,RA_10,"1,1,1,3,5,5,5-heptamethyltrisiloxane",217-496-1,1873-88-7,Mono-,203-497-4,Mono-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group
17,RA_18,"Alkenes, C10-14",288-213-7,85681-75-0,UVCB,203-893-7,Mono-,306-492-6,UVCB,248-205-6,UVCB,272-762-4,UVCB,204-012-9,UVCB,209-753-1,Mono-,Group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,RA_142,Tetraammonium ethylenediaminetetraacetate,245-022-3,22473-78-5,Mono-,205-358-3,Mono-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group
142,RA_143,Tetradec-1-ene,214-306-9,1120-36-1,UVCB,203-893-7,Mono-,306-492-6,UVCB,248-205-6,UVCB,272-762-4,UVCB,204-012-9,UVCB,209-753-1,Mono-,Group
144,RA_145,Tetrasodium ethylenediaminetetraacetate,200-573-9,64-02-8,Mono-,205-358-3,Mono-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group
145,RA_146,Triammonium hydrogen ethylenediaminetetraacetate,240-073-8,15934-01-7,Mono-,205-358-3,Mono-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group


In [334]:
test2 = (pd.melt(test, id_vars = ['index_key','Target_Substance', 'EC_List_Number', 'CASRN', 'Target_Substance_Type'], value_vars =['*Source_Substance_EC_1',
      '*Source_Substance_EC_2',
        '*Source_Substance_EC_3',
        '*Source_Substance_EC_4',
      '*Source_Substance_EC_5',
        '*Source_Substance_EC_6',
       ])
         .dropna(subset = 'value')
        )

In [335]:
test2[test2['Target_Substance'] == 'Cobalt']

,index_key,Target_Substance,EC_List_Number,CASRN,Target_Substance_Type,variable,value
1,RA_2,Cobalt,231-158-0,7440-48-4,Mono-,*Source_Substance_EC_1,231-589-4
150,RA_2,Cobalt,231-158-0,7440-48-4,Mono-,*Source_Substance_EC_2,215-157-2


In [336]:
test2.to_csv(external_dir/'ECHA_accepted_examples.csv')

In [348]:
test2.sort_values(by='index_key', kind='mergesort').to_csv(external_dir/'ECHA_accepted_examples.csv')

## Re-read in the cleaned up file that has been augmented with DSSTox and ECHA content

In [349]:
accepted_echa = pd.read_excel(external_dir/'ECHA_accepted_examples.xlsx')

In [360]:
accepted_echa_mono = (accepted_echa
 .query("Target_Substance_Type == 'Mono-' & target_dsstox_smiles.notnull() & source_dsstox_smiles.notnull()")
 .filter(['index_key', 'Target_Substance', 'Target_Substance_Type', 'target_dtxsid', 'target_dsstox_name',
       'target_dsstox_cas', 'target_dsstox_smiles','source_dtxsid', 'source_dsstox_name',
       'source_dsstox_cas', 'source_dsstox_smiles'])
)

In [361]:
accepted_echa_mono.to_csv(external_dir/'accepted_echa_discrete.csv')